# FP Growth
## Method
1.  Clean the data set
2.  Generate the table with each item and corresponding counts
3.  Generate FP tree
4.  Mine tree

### Import important libraries

In [11]:
import pandas as pd

In [12]:
transactions=[]
with open('Example_data.txt', 'r') as f:
    for line in f.readlines():
        L=line.split(',')
        L[-1]=L[-1].replace('\n','')
        transactions.append(L)
transactions[0]

['Private',
 '11th',
 'Never-married',
 'Machine-op-inspct',
 'Own-child',
 'Black',
 'Male',
 'United-States',
 '<=50K.']

In [13]:
from mlxtend.preprocessing import TransactionEncoder
te = TransactionEncoder()
te_ary = te.fit(transactions).transform(transactions)
df = pd.DataFrame(te_ary, columns=te.columns_)
#df.head()
from mlxtend.frequent_patterns import apriori

AP=apriori(df, min_support=0.6,use_colnames=True)
AP.sort_values('support',ascending=False).reset_index(drop=True)

,support,itemsets
0,0.900559,(United-States)
1,0.856581,(White)
2,0.790615,"(United-States, White)"
3,0.763774,(<=50K.)
4,0.688533,(Private)
5,0.684172,"(United-States, <=50K.)"
6,0.667035,(Male)
7,0.642221,"(<=50K., White)"
8,0.614827,"(United-States, Private)"


In [14]:
from mlxtend.preprocessing import TransactionEncoder
te = TransactionEncoder()
te_ary = te.fit(transactions).transform(transactions)
df = pd.DataFrame(te_ary, columns=te.columns_)
#df.head()
from mlxtend.frequent_patterns import fpgrowth
FP=fpgrowth(df, min_support=0.6,use_colnames=True)
FP.sort_values('support',ascending=False).reset_index(drop=True)

,support,itemsets
0,0.900559,(United-States)
1,0.856581,(White)
2,0.790615,"(United-States, White)"
3,0.763774,(<=50K.)
4,0.688533,(Private)
5,0.684172,"(United-States, <=50K.)"
6,0.667035,(Male)
7,0.642221,"(<=50K., White)"
8,0.614827,"(United-States, Private)"


In [15]:
transactions=pd.DataFrame({'TID':['T'+str(i) for i in range(100,1000,100)],'itemsets':[['I1','I2','I5'],['I2','I4'],['I2','I3'],['I1','I2','I4'],['I1','I3'],['I2','I3'],['I1','I3'],['I1','I2','I3','I5'],['I1','I2','I3']]})
transactions

,TID,itemsets
0,T100,"[I1, I2, I5]"
1,T200,"[I2, I4]"
2,T300,"[I2, I3]"
3,T400,"[I1, I2, I4]"
4,T500,"[I1, I3]"
5,T600,"[I2, I3]"
6,T700,"[I1, I3]"
7,T800,"[I1, I2, I3, I5]"
8,T900,"[I1, I2, I3]"


### Generate distinct items and corresponding counts
- The table will further used as header table
- input dataframe which contains all transactions<br>
        trans: |TID|itemsets|<br>
        min_support: eg. 0.5
- output an ordered dictionary with unique items and corresponding counts<br>
dataset: 

In [16]:
#generate the first set with correponding counts. Sort by the counts from highest to lowest. 
def GenC1(trans,min_support):
    C1={}
    for i in trans:
        for ii in set(i):
            if ii not in C1:
                C1[ii]=1
            else:
                C1[ii]+=1
    C1={key:value for key, value in C1.items() if value/len(trans)>=min_support}#include min_support
    return {i:[k,None] for i,k in sorted(C1.items(),key=lambda x:x[1],reverse=True)}


In [17]:
C1=GenC1(transactions['itemsets'],0.0)
C1

{'I2': [7, None],
 'I1': [6, None],
 'I3': [6, None],
 'I5': [2, None],
 'I4': [2, None]}

If C1 does not contain any value, there is no need to further process

### Generate FP tree
- input dataset with all transactiosn
- output an FPtree

In [18]:
#tree node
class TreeNode:
    def __init__(self, item, count, parentNode):
        self.item=item
        self.count=count
        self.nodelink=None
        self.parent=parentNode
        self.children={}
    def inc_count(self,num):
        self.count+=num
    #def disp(self,ind=1):
    #    print('   '*ind, self.item,'  ',self.count)
    #    for child in self.children.values():
    #        child.disp(ind+1)

In [19]:
def link_same_items(curr,targetNode):
    while curr.nodelink is not None:
        curr=curr.nodelink
    curr.nodelink=targetNode

def updateTree_Table(SortedItems,FPTree,headertable,count):
    if SortedItems[0] in FPTree.children:
        FPTree.children[SortedItems[0]].inc_count(count)
    else:
        FPTree.children[SortedItems[0]]=TreeNode(SortedItems[0],count,FPTree)
        if headertable[SortedItems[0]][1]==None:
            headertable[SortedItems[0]][1]=FPTree.children[SortedItems[0]]
        else:
            link_same_items(headertable[SortedItems[0]][1], FPTree.children[SortedItems[0]])
    if len(SortedItems)>1:
        updateTree_Table(SortedItems[1:],FPTree.children[SortedItems[0]],headertable,count)

In [24]:
#Build Tree
def Build_tree(trans,headertable,count=1):
    recTree=TreeNode('Null',1,None)
    for itemset in trans:
        #order the itemset according to the header table
        if len(itemset)>0:
            print(list(itemset))
            if len(itemset)>1:
                SortedItems=sorted(list(itemset),key=lambda x:headertable[x][0],reverse=True)
            else:
                SortedItems=sorted(itemset,key=lambda x:headertable[x][0],reverse=True)
            updateTree_Table(SortedItems,recTree,headertable,count)
    return recTree,headertable


FPtree, HeaderTable=Build_tree(transactions['itemsets'],C1,1)

['I1', 'I2', 'I5']
['I2', 'I4']
['I2', 'I3']
['I1', 'I2', 'I4']
['I1', 'I3']
['I2', 'I3']
['I1', 'I3']
['I1', 'I2', 'I3', 'I5']
['I1', 'I2', 'I3']


### Print Tree

In [25]:
print(FPtree)

In [12]:
#print(FPTree.children.keys())
def printTree(Tree,i=0,p='null'):
    if len(Tree.children)==0:
        print('GET BACK')
        return
    else:
        i+=1
        print(i,':',p,':',Tree.item,Tree.children.keys())
        for key,child in Tree.children.items():
            printTree(child,i,key)
printTree(FPtree)
print(pd.DataFrame(HeaderTable.values(),HeaderTable.keys()))

1 : null : Null dict_keys(['I2', 'I1'])
2 : I2 : I2 dict_keys(['I1', 'I4', 'I3'])
3 : I1 : I1 dict_keys(['I5', 'I4', 'I3'])
GET BACK
GET BACK
4 : I3 : I3 dict_keys(['I5'])
GET BACK
GET BACK
GET BACK
2 : I1 : I1 dict_keys(['I3'])
GET BACK
    0                                          1
I2  7  <__main__.TreeNode object at 0x114b88cc0>
I1  6  <__main__.TreeNode object at 0x114b88d30>
I3  6  <__main__.TreeNode object at 0x1218fc8d0>
I5  2  <__main__.TreeNode object at 0x114b88048>
I4  2  <__main__.TreeNode object at 0x114b88518>


### Mine Tree

In [26]:
def ascendTree(leafNode,PrefixPath):
    if leafNode.parent is not None:
        PrefixPath.append(leafNode.item)
        ascendTree(leafNode.parent, PrefixPath)

def findPrefixPath(baseitem,TreeNode):
    CPB={}
    while TreeNode is not None:
        PrefixPath=[]
        ascendTree(TreeNode,PrefixPath)
        if len(PrefixPath)>1:
            CPB[frozenset(PrefixPath[1:])]=TreeNode.count
        TreeNode=TreeNode.nodelink
    return CPB

In [27]:
#FPtree
def mineTree(inTree, headertable, min_support,preFix, freqItemList):
    reverse_headertable = [v for v,_ in sorted(headertable.items(), key=lambda x: x[1][0])]
    for baseitem in reverse_headertable:
        #Find prefixed items in a set
        newFreqSet=preFix.copy()
        newFreqSet.add(baseitem)
        freqItemList.append(newFreqSet)

        #Conditional Pattern Base
        CPB=findPrefixPath(baseitem,headertable[baseitem][1])
        if len(CPB)!=0:
            print(pd.DataFrame({'Item':[str(baseitem) for i in range(len(CPB))],'Prefix':[(i) for i in CPB]}))
        #print(baseitem,CPB)
        #Make conditional FP tree accordingly
        CPBC1=GenC1(CPB,min_support)
        if len(CPBC1)!=0:
            print(CPBC1)
            print('-'*30)
        CondFPtree, Condheadertable=Build_tree(list(CPB.keys()),CPBC1,1)

        if Condheadertable is not None:
            mineTree(CondFPtree, Condheadertable, min_support,newFreqSet, freqItemList)
    
headertable=HeaderTable
freqItemList=[]
mineTree(FPtree, headertable, 3, set([]), freqItemList)


  Item        Prefix
0   I5      (I1, I2)
1   I5  (I3, I1, I2)
['I1', 'I2']


KeyError: 'I1'

In [169]:
trans=list(CPB.keys())
headertable=CPBC1
count=1
Build_tree(list(CPB.keys()),CPBC1,1)

recTree=TreeNode('Null',1,None)
for itemset in trans:
    #order the itemset according to the header table
    if len(itemset)>0:
        SortedItems=sorted(list(itemset),key=lambda x:headertable[x][0],reverse=True)
    updateTree_Table(SortedItems,recTree,headertable,count)


### calculate rules

In [ ]:
def aprioriGen(Lk, k):
    retList = []
    lenLk = len(Lk)
    for i in range(lenLk):
        for j in range(i+1, lenLk):
            L1 = list(Lk[i])[:k-2]; L2 = list(Lk[j])[:k-2]
            L1.sort(); L2.sort()
            if L1 == L2: 
                retList.append(Lk[i] | Lk[j])
    return retList

def calcConf(freqSet, H, supportData, br1, minConf=0.7):
    prunedH = []
    for conseq in H:
        conf = supportData[freqSet] / supportData[freqSet - conseq]
        if conf >= minConf:
            print "{0} --> {1} conf:{2}".format(freqSet - conseq, conseq, conf)
            br1.append((freqSet - conseq, conseq, conf))
            prunedH.append(conseq)
    return prunedH

def rulesFromConseq(freqSet, H, supportData, br1, minConf=0.7):
    m = len(H[0])
    if len(freqSet) > m+1:
        Hmp1 = aprioriGen(H, m+1)
        Hmp1 = calcConf(freqSet, Hmp1, supportData, br1, minConf)
        if len(Hmp1)>1:
            rulesFromConseq(freqSet, Hmp1, supportData, br1, minConf)

def generateRules(freqItemList, supportData, minConf=0.7):
    bigRuleList = []
    for freqSet in freqItemList:
        H1 = [frozenset([item]) for item in freqSet]
        if len(freqSet)>1:
            rulesFromConseq(freqSet, H1, supportData, bigRuleList, minConf)
        else:
            calcConf(freqSet, H1, supportData, bigRuleList, minConf)
    return bigRuleList

In [ ]:
def calSuppData(headerTable, freqItemList, total):
    suppData = {}
    for Item in freqItemList:
        # 找到最底下的结点
        Item = sorted(Item, key=lambda x:headerTable[x][0])
        base = findPrefixPath(Item[0], headerTable)
        # 计算支持度
        support = 0
        for B in base:
            if frozenset(Item[1:]).issubset(set(B)):
                support += base[B]
        # 对于根的儿子，没有条件模式基
        if len(base)==0 and len(Item)==1:
            support = headerTable[Item[0]][0]
            
        suppData[frozenset(Item)] = support/float(total)
    return suppData


suppData = fpgrowth.calSuppData(myHeaderTab, freqItems, len(parsedDat))
suppData[frozenset([])] = 1.0
for x,v in suppData.iteritems():
    print x,v

freqItems = [frozenset(x) for x in freqItems]
fpgrowth.generateRules(freqItems, suppData)

In [167]:
printTree(recTree)

1 : null : Null dict_keys(['I1'])
2 : I1 : I1 dict_keys(['I2'])
3 : I2 : I2 dict_keys(['I3'])
GET BACK


In [164]:
print(list(CPB.keys()))
for i in CPB.keys():
    print(i)
print(CPBC1)
itemset=list(CPB.keys())

[frozenset({'I1', 'I2'}), frozenset({'I1', 'I2', 'I3'})]
frozenset({'I1', 'I2'})
frozenset({'I1', 'I2', 'I3'})
{'I1': [2, None], 'I2': [2, None], 'I3': [1, None]}


In [83]:
pd.DataFrame.from_dict(CPB,orient='index').reset_index()

,index,0
0,"(I1, I2)",1
1,"(I1, I2, I3)",1
